In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import math

from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import cv2 as cv
from PIL import Image

import multiprocessing
from multiprocessing.pool import ThreadPool

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer

import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

import cv2

def preProcessImageObsolete(image, cutoff=127, maxContours=5):
    image = np.uint8(image)
    im = np.uint8(image)
    red, thresh = cv2.threshold(im, cutoff, 255, 0)
    im2, contours, hierarchy= cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.uint8(np.ones(im.shape))
    largest_contours = sorted(contours, key=cv2.contourArea)
    for ind, contour in enumerate(largest_contours[maxContours:]):
        mask = cv2.drawContours(mask, [largest_contours[ind]], -1, 0, -1)
        
    filteredImage = cv2.bitwise_and(thresh, thresh, mask=mask)
    ## plt.imshow(filteredImage)
    return filteredImage

def preprocess(image, cutoff=127, maxContours=5):
    image = np.uint8(image)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image,connectivity = 4)
    sizes = stats[:,-1]
    max_label = 1
    max_size = sizes[1]
    for i in range(2,nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
    img = np.zeros(output.shape)
    img[output == max_label] = 255
    return img

def preProcessImage(image, cutoff=127, maxContours=10):
    image = np.uint8(image)
    im = np.uint8(image)
    red, thresh = cv2.threshold(im, cutoff, 255, 0)
    im2, contours, hierarchy= cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros(im.shape, np.uint8)
    largest_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    for ind, contour in enumerate(largest_contours[:5]):
        x, y, w, h = cv2.boundingRect(contour)
        mask[y:y+h, x:x+w] = 255    
    filteredImage = cv2.bitwise_and(thresh, thresh, mask=mask)
    #plt.imshow(filteredImage)
    #plt.figure()
    #plt.imshow(thresh)
    #plt.figure()
    #plt.imshow(mask)
    return filteredImage.reshape((image.shape))



Using TensorFlow backend.


In [ ]:
data = np.load("all/train_images.npy",encoding='bytes')

x = []
for image in data:
    image = image[1].reshape(100,100)
#     image = preprocess(image)
    x.append(image)

x_pre = []
for image in data:
    image = image[1].reshape(100,100)
    x_pre.append(image)



labels = pd.read_csv("all/train_labels.csv")
y = []
for i in range(len(labels)):
    label = labels['Category'][i]
    y.append(label)


x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size = 0.1, random_state=30)

x_train_backup = x_train
x_validation_backup = x_validation

x_train = np.array(x_train).reshape(len(x_train), 100, 100, 1).astype('float32') / 255
x_validation = np.array(x_validation).reshape(len(x_validation), 100, 100, 1).astype('float32') / 255

encoder = LabelBinarizer()

y_train = encoder.fit_transform(y_train)
y_validation = encoder.fit_transform(y_validation)

y_train_decoded = encoder.inverse_transform(y_train)
y_validation_decoded = encoder.inverse_transform(y_train)


##Setup CNN
model = Sequential()
# model.add(Conv2D(filters=20,
#                  kernel_size=(4, 4),
#                  padding='same',
#                  input_shape=(100, 100, 1),
#                  activation='relu'))
# model.add(Conv2D(filters=20,
#                  kernel_size=(4, 4),
#                  padding='same',
#                  activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Conv2D(filters=80,
#                  kernel_size=(4, 4),
#                  padding='same',
#                  activation='relu'))
# model.add(Conv2D(filters=80,
#                  kernel_size=(3, 3),
#                  padding='same',
#                  activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Conv2D(filters=80,
#                  kernel_size=(3, 3),
#                  padding='same',
#                  activation='relu'))
# model.add(Conv2D(filters=80,
#                  kernel_size=(3, 3),
#                  padding='same',
#                  activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Dropout(0.1))
# model.add(Flatten())
# model.add(Dense(units=512))
# model.add(BatchNormalization())
# model.add(Dense(units=256))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.25))
# model.add(Dense(31, activation='softmax'))



# model.add(Conv2D(filters=32,
#                 kernel_size=(5,5), 
#                 padding='same',
#                 activation='relu',
#                 input_shape=(100, 100, 1)))
# # Add this for iteration 3
# #BatchNormalization(axis=-1)
# model.add(Conv2D(filters=32,
#                 kernel_size=(5,5),
#                 activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# #BatchNormalization(axis=-1)
# #model.add(Dropout(0.25))

# model.add(Conv2D(filters=64,
#                 kernel_size=(3,3),
#                 padding='Same',
#                 activation='relu'))
# #BatchNormalization(axis=-1)
# model.add(Conv2D(filters=64,
#                 kernel_size=(3,3),
#                 padding='Same',
#                 activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# #model.add(Dropout(0.25))

# model.add(Conv2D(filters=128,
#                 kernel_size=(5,5),
#                 padding='Same',
#                 activation='relu'))
# #BatchNormalization(axis=-1)
# model.add(Conv2D(filters=128,
#                 kernel_size=(5,5),
#                 padding='Same',
#                 activation='relu'))

# model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# #model.add(Dropout(0.25))

# model.add(Flatten())
# BatchNormalization(axis=-1)
# model.add(Dropout(0.5))

# model.add(Dense(512, activation='relu'))
# BatchNormalization(axis=-1)
# model.add(Dropout(0.5))
# # model.add(Dense(1024, activation='relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(31, activation='softmax'))
input_shape=(100, 100, 1)
model = Sequential([
    

        Conv2D(32, strides=(1, 1),kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(32, strides=(1, 1),kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2),strides=(1, 1)),
        Dropout(0.25),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),

        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5)])
model.add(Dense(31, activation='softmax'))


optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #adam

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

##Fit CNN model
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images
datagen.fit(x_train)




learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)




batch_size = 32
epochs = 50

train_history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    epochs=epochs, validation_data=(x_validation, y_validation),
                                    verbose=2, steps_per_epoch=x_train.shape[0] // batch_size
                                    , callbacks=[learning_rate_reduction])


plt.plot(train_history.history['acc'])
plt.plot(train_history.history['val_acc'])
epoch_num = len(train_history.epoch)
final_epoch_train_acc = train_history.history['acc'][epoch_num - 1]
final_epoch_validation_acc = train_history.history['val_acc'][epoch_num - 1]
plt.text(epoch_num, final_epoch_train_acc, 'train = {:.3f}'.format(final_epoch_train_acc))
plt.text(epoch_num, final_epoch_validation_acc-0.01, 'valid = {:.3f}'.format(final_epoch_validation_acc))
plt.title('Train History')
plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.xlim(xmax=epoch_num+1)
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


data_test = np.load("all/test_images.npy",encoding='bytes')

x_test = []
for image in data_test:
    image = image[1].reshape(100,100)
#     image = preprocess(image)
    x_test.append(image)

x_test = np.array(x_test).reshape(len(x_test), 100, 100, 1).astype('float32') / 255

prediction = model.predict_classes(x_test)
df = pd.DataFrame(prediction)
df.index += 1
df.index.name = 'Id'
df.columns = ['Category']
df.to_csv('cnn5.csv', header=True)

word_class = []
for i in range(len(df)):
    c = encoder.classes_[df['Category'][i+1]]
    word_class.append(c)

word_class = np.array(word_class)
word_class = pd.DataFrame(word_class)
word_class.columns = ['Category']
word_class.to_csv('cnn5.csv', header=True)


Epoch 1/50
 - 40s - loss: 4.2566 - acc: 0.0516 - val_loss: 4.1148 - val_acc: 0.0450
Epoch 2/50
 - 34s - loss: 3.6618 - acc: 0.0677 - val_loss: 3.8141 - val_acc: 0.0690
Epoch 3/50
 - 34s - loss: 3.3974 - acc: 0.0901 - val_loss: 3.7402 - val_acc: 0.0570
Epoch 4/50
 - 34s - loss: 3.2552 - acc: 0.0934 - val_loss: 3.0012 - val_acc: 0.1240
Epoch 5/50
 - 34s - loss: 3.1240 - acc: 0.1095 - val_loss: 3.5823 - val_acc: 0.0630
Epoch 6/50
